# Predicted Validation Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Validation Data
'''
Loading Validation Data for which Coarse Grained Model 
(NonHostile vs Hostile Classification) predicted Hostile Class
'''
file = '/content/drive/My Drive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Dataset/hostile_validate_pred.xlsx'
test_df = pd.read_excel(file)

# Data Preparation into Pandas Dataframe for Model Input

def get_data(a):
  Unique_ID = list(a['Unique ID'])
  sentence = list(a['Post'])
  text_labels = list(a['Labels Set'])

  label = []
  for i in text_labels:
    if i=='hostile':
        label.append(0)

  raw_data_train = {'UID':Unique_ID,'sentence': sentence, 'label': label}
  df = pd.DataFrame(raw_data_train, columns = ['UID','sentence','label'])
  return df

test_data  = get_data(test_df)

print(test_data[0:3])

# Hostile ID Collection 

In [ ]:
# Collecting Hostile IDs from Validation Data

data = test_data

hos_ids = []
for i in range(len(data)):
  id = data['UID'][i]
  hos_ids.append(id)

hos_ids = np.array(hos_ids, dtype=np.int)

# Predicted Labels Loading (Fake, Hate, Offensive, Defamation)

In [ ]:
d_lab = np.load('/content/Final_Defamation_validation_Pred_Label.npy', allow_pickle=True)
f_lab = np.load('/content/Final_Fake_validation_Pred_Label.npy', allow_pickle=True)
h_lab = np.load('/content/Final_Hate_validation_Pred_Label.npy', allow_pickle=True)
o_lab = np.load('/content/Final_Offensive_validation_Pred_Label.npy', allow_pickle=True)

# Merging Labels to Asses Performance measurement

In [ ]:
# Merging Predicted labels into a single numpy array
# Reference: [non_hostile,defamation,fake,hate,offensive]

predicted_labels = []

count = 0
for i in range(1,812):
  row = []
  if i not in hos_ids:
    row.append([1,0,0,0,0])
  else:
    alt_row = [0,0,0,0,0]
    if d_lab[count]==1:
      alt_row[1] = 1
    if f_lab[count]==1:
      alt_row[2] = 1
    if h_lab[count]==1:
      alt_row[3] = 1
    if o_lab[count]==1:
      alt_row[4] = 1
    count += 1
    row.append(alt_row)
  predicted_labels.append(row)

pred_lab = np.reshape(np.array(predicted_labels),(811,5)) # Final Predicted Labels
np.save('Final_Pred_Labels_Validation.npy',pred_lab)

# Loading True Labels (811,5) and Predicted Labels (811,5) for Final Result Calculation

In [ ]:
y_true = np.load('/content/True_Validation_labels.npy', allow_pickle=True)       # Provided by Organizers
y_pred = np.load('/content/Final_Pred_Labels_Validation.npy', allow_pickle=True)

# Computing Scores

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_true, y_pred, average='macro'))
print(f1_score(y_true, y_pred, average='micro'))
print(f1_score(y_true, y_pred, average='weighted'))

# Creating Final Submission File

In [ ]:
# Reference: [non_hostile,defamation,fake,hate,offensive]
labels = []

for i in range(1,y_pred.shape[0]+1):       
  idx = np.argwhere(y_pred[i]>0)
  idx = idx.reshape(idx.shape[0],)
  if (len(idx)==0):
    lab_text.append('defamation')
  else:
  for j in idx:
    if j==0:
      lab_text.append('non-hostile')
    if j==1:
      lab_text.append('defamation')
    if j==2:
      lab_text.append('fake')
    if j==3:
      lab_text.append('hate')
    if j==4:
      lab_text.append('offensive')
      
labels.append(lab_text)

def final_submission(label_list):
  import csv
  data = []
  titles = ['Unique ID','Labels Set']
  data.append(titles)
  for i in range(len(label_list)):
    row = []
    row.append(i+1)
    lab_text = ''
    for j in range(len(label_list[i])):
      lab_text += str(label_list[i][j])+','
    lab_text = lab_text[:-1]+''
    row.append(str(lab_text))
    data.append(row)

  file1 = "answer.csv"
  with open(file1, 'w') as csvfile:  
    csvwriter = csv.writer(csvfile)   
    csvwriter.writerows(data)

final_submission(labels)

# Score with Official Evaluation Script

In [ ]:
### Order of Labels --> [Hostile,defamation,fake,hate,offensive,non-hostile]
### An example      --> [1,0,1,1,0,0]


import numpy as np
import pandas as pd
from sklearn.metrics import f1_score



def preprocess(df):
    
    df = df.dropna()
    
    df.insert(len(df.columns)-1,'Hostile', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Defamation', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Fake', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Hate', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Offensive', np.zeros(len(df),dtype=int))
    df.insert(len(df.columns)-1,'Non-Hostile', np.zeros(len(df),dtype=int))    
    
    for i in range(len(df)):
        text = df['Labels Set'][i]
        text = text.lower()
        text = text.replace('\n',"")
        text = text.replace('"',"")
        text = text.replace(" ","")
        text = text.split(',')


        for word in text:
            if word == 'defamation':
                df.at[i,'Hostile']    = 1
                df.at[i,'Defamation'] = 1
    
            if word == 'fake':
                df.at[i,'Hostile']    = 1
                df.at[i,'Fake'] = 1
    
            if word == 'hate':
                df.at[i,'Hostile']    = 1
                df.at[i,'Hate'] = 1
    
            if word == 'offensive':
                df.at[i,'Hostile']    = 1
                df.at[i,'Offensive'] = 1
    
            if word == 'non-hostile' and df['Hostile'][i]==0:
                df.at[i,'Hostile']    = 0
                df.at[i,'Non-Hostile'] = 1

    return df 
  
    
def get_scores(y_true, y_pred):
    
    hostility_true = y_true['Hostile']
    hostility_pred = y_pred['Hostile']
    
    hostility_f1 = f1_score(y_true=hostility_true, y_pred=hostility_pred, average='weighted')
    
    
    nh_indexes = y_true[y_true['Hostile']==0].index
    y_true = y_true.drop(nh_indexes)
    y_true = y_true.reset_index(drop=True)
    
    y_pred = y_pred.drop(nh_indexes)
    y_pred = y_pred.reset_index(drop=True)
    
    
    fine_true = y_true[['Defamation','Fake','Hate','Offensive']]
    fine_pred = y_pred[['Defamation','Fake','Hate','Offensive']]
    
    
    fine_f1          = f1_score(y_true=fine_true, y_pred=fine_pred, average=None)
    defame_f1        = fine_f1[0]
    fake_f1          = fine_f1[1]
    hate_f1          = fine_f1[2]
    offensive_f1     = fine_f1[3]
    weighted_fine_f1 = f1_score(y_true=fine_true, y_pred=fine_pred, average='weighted')

    return [hostility_f1, defame_f1, fake_f1, hate_f1, offensive_f1, weighted_fine_f1]

ground_truth_path      = 'content\Validation Ground Truth.csv'
submission_file_path   = 'content\answer.csv"

try:  
    y_true = pd.read_csv(ground_truth_path)
    y_pred = pd.read_csv(submission_file_path)
    
    y_true = preprocess(y_true)
    y_pred = preprocess(y_pred)
    
    team_score = get_scores(y_true,y_pred)
    
    
except:
    team_score = [0,0,0,0,0,0]
    
        
print("Coarse Grained F1-score: ", team_score[0])
print("Defamation F1-score:     ", team_score[1])
print("Fake F1-score:           ", team_score[2])
print("Hate F1-score:           ", team_score[3])
print("Offensive F1-score:      ", team_score[4])
print("Fine Grained F1-score:   ", team_score[5])